get the number of egenes and number sgenes that we find in all tissues (for ASHG abstract)

In [2]:
import pandas as pd
import glob
import os
from tqdm.notebook import tqdm

which genes are new?

In [3]:
# pull in the gene ids
gencode_ids = pd.read_csv('/home/klawren/oak/gtex/data/other_references/gencode/gencode_id_matching_simplified.txt', sep='\t', index_col=0)
gencode_ids = gencode_ids[~(gencode_ids['gene_type_grouped'] == 'artifact')]
gencode_ids.head()

,gene_id_stable,gene_id_v47,gene_id_v39,gene_type_grouped,gene_type_grouped_highlight,in_v47_full,in_v47_collapsed,in_v39_full,in_v39_collapsed,collapsed_location_change_v47,collapsed_location_change_v39,collapsed_location_change_between_versions,full_location_change_between_versions
0,ENSG00000290825,ENSG00000290825.2,NaN,lncRNA,CLS_lncRNA,True,True,False,False,True,False,True,True
1,ENSG00000223972,ENSG00000223972.6,ENSG00000223972.5,pseudogene,pseudogene,True,True,True,True,True,True,True,True
2,ENSG00000243485,ENSG00000243485.6,ENSG00000243485.5,lncRNA,other_lncRNA,True,True,True,True,False,True,True,True
3,ENSG00000308361,ENSG00000308361.1,NaN,lncRNA,CLS_lncRNA,True,True,False,False,False,False,True,True
4,ENSG00000290826,ENSG00000290826.2,NaN,lncRNA,CLS_lncRNA,True,True,False,False,False,False,True,True


In [4]:
new_lncRNA_filter = ((gencode_ids['gene_type_grouped'] == 'lncRNA') & ~(gencode_ids['in_v39_full']))
all_lncRNA_filter = ((gencode_ids['gene_type_grouped'] == 'lncRNA') & (gencode_ids['in_v47_full']))
print('There are {} lncRNAs in v47, {} of them are not in v39'.format(sum(all_lncRNA_filter), sum(new_lncRNA_filter)))

There are 34914 lncRNAs in v47, 18307 of them are not in v39


how many egenes do we find?

In [5]:
# load in all eGene summary files
egene_pattern = os.path.join('/home/klawren/oak/gtex/output/all_tissues/eqtls', '*.v11.eGenes.txt.gz')
egene_files = glob.glob(egene_pattern)

egenes = []
for egene_file in tqdm(egene_files):
    tissue_egene_df = pd.read_csv(egene_file, sep='\t')
    tissue_egene_df['tissue_id'] = egene_file.split('/')[-1].split('.v11')[0]
    egenes.append(tissue_egene_df)
    
egenes = pd.concat(egenes)

  0%|          | 0/29 [00:00<?, ?it/s]

In [6]:
# how many were tested?
gencode_ids['tested_egene'] = gencode_ids['gene_id_v47'].isin(egenes['gene_id'])

print('{} ({:.2f}%) new lncRNAs were tested as possible eGenes'.format(gencode_ids[new_lncRNA_filter]['tested_egene'].sum(), gencode_ids[new_lncRNA_filter]['tested_egene'].mean()*100))
print('{} ({:.2f}%) all lncRNAs were tested as possible eGenes'.format(gencode_ids[all_lncRNA_filter]['tested_egene'].sum(), gencode_ids[all_lncRNA_filter]['tested_egene'].mean()*100))

8612 (47.04%) new lncRNAs were tested as possible eGenes
20180 (57.80%) all lncRNAs were tested as possible eGenes


In [7]:
# we use a 5% FDR for eGenes
signifigant_egenes = egenes[egenes['qval'] < 0.05]
gencode_ids['is_egene'] = gencode_ids['gene_id_v47'].isin(signifigant_egenes['gene_id'])

# how many lncRNAs are signifigant?
print('{} ({:.2f}%) new lncRNAs are eGenes'.format(gencode_ids[new_lncRNA_filter]['is_egene'].sum(), gencode_ids[new_lncRNA_filter]['is_egene'].mean()*100))
print('{} ({:.2f}%) all lncRNAs are eGenes'.format(gencode_ids[all_lncRNA_filter]['is_egene'].sum(), gencode_ids[all_lncRNA_filter]['is_egene'].mean()*100))

6106 (33.35%) new lncRNAs are eGenes
15716 (45.01%) all lncRNAs are eGenes


how many sgenes do we find?

In [8]:
# load in all eGene summary files
sgene_pattern = os.path.join('/home/klawren/oak/gtex/output/all_tissues/sqtls', '*.v11.sGenes.txt.gz')
sgene_files = glob.glob(sgene_pattern)

sgenes = []
for sgene_file in tqdm(sgene_files):
    tissue_sgene_df = pd.read_csv(sgene_file, sep='\t')
    tissue_sgene_df['tissue_id'] = sgene_file.split('/')[-1].split('.v11')[0]
    sgenes.append(tissue_sgene_df)
sgenes = pd.concat(sgenes)

  0%|          | 0/29 [00:00<?, ?it/s]

In [9]:
# how many were tested?
gencode_ids['tested_sgene'] = gencode_ids['gene_id_v47'].isin(sgenes['gene_id'])

print('{} ({:.2f}%) new lncRNAs were tested as possible sGenes'.format(gencode_ids[new_lncRNA_filter]['tested_sgene'].sum(), gencode_ids[new_lncRNA_filter]['tested_sgene'].mean()*100))
print('{} ({:.2f}%) all lncRNAs were tested as possible sGenes'.format(gencode_ids[all_lncRNA_filter]['tested_sgene'].sum(), gencode_ids[all_lncRNA_filter]['tested_sgene'].mean()*100))

1987 (10.85%) new lncRNAs were tested as possible sGenes
6056 (17.35%) all lncRNAs were tested as possible sGenes


In [10]:
# we use a 5% FDR for sGenes
signifigant_sgenes = sgenes[sgenes['qval'] < 0.05]
gencode_ids['is_sgene'] = gencode_ids['gene_id_v47'].isin(signifigant_sgenes['gene_id'])

# how many lncRNAs are signifigant?
print('{} ({:.2f}%) new lncRNAs are sGenes'.format(gencode_ids[new_lncRNA_filter]['is_sgene'].sum(), gencode_ids[new_lncRNA_filter]['is_sgene'].mean()*100))
print('{} ({:.2f}%) all lncRNAs are sGenes'.format(gencode_ids[all_lncRNA_filter]['is_sgene'].sum(), gencode_ids[all_lncRNA_filter]['is_sgene'].mean()*100))

887 (4.85%) new lncRNAs are sGenes
2773 (7.94%) all lncRNAs are sGenes
